In [2]:
import optuna

In [3]:
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

In [5]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2023-01-12 11:06:35,015] A new study created in memory with name: no-name-828d101f-07f9-4e98-ac21-fe6864255c59
[I 2023-01-12 11:06:35,018] Trial 0 finished with value: 12.225726049689788 and parameters: {'x': -1.4965305732525476}. Best is trial 0 with value: 12.225726049689788.
[I 2023-01-12 11:06:35,019] Trial 1 finished with value: 32.6142912394491 and parameters: {'x': -3.7108923330289727}. Best is trial 0 with value: 12.225726049689788.
[I 2023-01-12 11:06:35,021] Trial 2 finished with value: 37.6504537326583 and parameters: {'x': 8.135996555789312}. Best is trial 0 with value: 12.225726049689788.
[I 2023-01-12 11:06:35,022] Trial 3 finished with value: 9.000771720975646 and parameters: {'x': 5.000128617405535}. Best is trial 3 with value: 9.000771720975646.
[I 2023-01-12 11:06:35,024] Trial 4 finished with value: 0.11626805826449818 and parameters: {'x': 1.6590189766797891}. Best is trial 4 with value: 0.11626805826449818.
[I 2023-01-12 11:06:35,025] Trial 5 finished with value

[I 2023-01-12 11:06:35,223] Trial 45 finished with value: 2.2468338282475813 and parameters: {'x': 3.498944237871303}. Best is trial 42 with value: 0.0001742017832555057.
[I 2023-01-12 11:06:35,229] Trial 46 finished with value: 10.034498051910713 and parameters: {'x': -1.1677275848643793}. Best is trial 42 with value: 0.0001742017832555057.
[I 2023-01-12 11:06:35,234] Trial 47 finished with value: 0.014602000228052506 and parameters: {'x': 1.8791612635449522}. Best is trial 42 with value: 0.0001742017832555057.
[I 2023-01-12 11:06:35,239] Trial 48 finished with value: 1.3818633276757557 and parameters: {'x': 3.1755268298408827}. Best is trial 42 with value: 0.0001742017832555057.
[I 2023-01-12 11:06:35,244] Trial 49 finished with value: 6.467974491381204 and parameters: {'x': 4.543221282425343}. Best is trial 42 with value: 0.0001742017832555057.
[I 2023-01-12 11:06:35,250] Trial 50 finished with value: 14.99743574260016 and parameters: {'x': 5.8726522878513325}. Best is trial 42 with

[I 2023-01-12 11:06:35,488] Trial 90 finished with value: 114.83752151897535 and parameters: {'x': -8.716227018824085}. Best is trial 71 with value: 3.126118683351869e-05.
[I 2023-01-12 11:06:35,496] Trial 91 finished with value: 0.0030542440489279583 and parameters: {'x': 1.9447347844577807}. Best is trial 71 with value: 3.126118683351869e-05.
[I 2023-01-12 11:06:35,501] Trial 92 finished with value: 0.38280001067546954 and parameters: {'x': 2.6187083405575438}. Best is trial 71 with value: 3.126118683351869e-05.
[I 2023-01-12 11:06:35,509] Trial 93 finished with value: 0.0008322620618428956 and parameters: {'x': 1.971151047474078}. Best is trial 71 with value: 3.126118683351869e-05.
[I 2023-01-12 11:06:35,517] Trial 94 finished with value: 48.667004683261105 and parameters: {'x': 8.976174072029819}. Best is trial 71 with value: 3.126118683351869e-05.
[I 2023-01-12 11:06:35,523] Trial 95 finished with value: 0.24322379854654474 and parameters: {'x': 1.5068227513899848}. Best is trial 

In [6]:
study.best_params

{'x': 2.005591170435027}

# lgbm + optuna

In [10]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import numpy as np

def objective(trial, X, y):
    # 后面填充
    param_grid = {}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = preds

    return np.mean(cv_scores)

In [13]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    print(X_train, X_test)
        
# cv_scores = np.empty(5)

NameError: name 'X' is not defined

In [ ]:
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplgb

def objective(trial):
    X_train, X_test, y_train, y_test=train_test_split(data, target, train_size=0.3)# 数据集划分
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17, 20, 50]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('cat_smooth', 1, 100)      
    }
    
    lgb=LGBMRegressor(**param)
    lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)
    pred_lgb=lgb.predict(X_test)
    rmse = mean_squared_error(y_test, pred_lgb, squared=False)
    return rmse


# project

In [ ]:
from gaia.metrics import symmetric_mean_absolute_percentage_error as smape, forecast_bias as kpi_fb
import optuna
def objective(trial):
    model_params = {
        'boosting_type': 'gbdt',
        'objective': trial.suggest_categorical('objective', ['quantile']),
        'alpha':trial.suggest_float("alpha", 0.4, 0.95),
        'num_leaves': trial.suggest_int("num_leaves", 2, 256),
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.5),
        'n_estimators': trial.suggest_int("n_estimators", 100, 700, 50),
        'metric': 'quantile',
        'feature_fraction': trial.suggest_float("feature_fraction", 0.4, 1.0),
        'bagging_fraction': trial.suggest_float("bagging_fraction", 0.4, 1.0),
        'max_bin': trial.suggest_int("max_bin", 100, 250, 10),
        'max_depth': trial.suggest_int("max_depth", 7, 15, 1),
        'reg_alpha': trial.suggest_float("reg_alpha", 1e-8, 10.0),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 256),
        'min_data_in_bin': trial.suggest_int('min_data_in_bin', 1, 256),
        'min_gain_to_split' : trial.suggest_discrete_uniform('min_gain_to_split', 0.1, 5, 0.01),      
        'n_jobs':16,
    }
    brand_cate = [
                ("COMFORT", "FABRIC ENHANCERS")
                 ]
    res_df_list = []
    features_list = []
    models = []
    cate_feats_list = []
    
    for brand, category in brand_cate:
        for fcst_date in range(202115, 202120):
            full_df = make_full_df(weekly_sales, fcstlist, fcst_date, 9, category, brand )
            merge_df = data_prepare(full_df, barcode_master, store_master, promotion, calendar_features, wm_yr_wk_week, new_agg_holiday_df, fcst_date)
            processed_df = data_process(merge_df, brand)

    #         processed_df = processed_df.rename(columns={'year_week':'wm_yr_wk', 'prom_type':'promo_type'})
            for horizon in range(1, 10):
                print('%%%%%% horizon start:', horizon)
                feature_df = features_create(processed_df, horizon, used_weekend_sales=False)
                print('%%%%%% feature done, and encoding start')
                encoded_df, features, cate_feats = features_encoding(feature_df, fcst_date)
                print('%%%%%% encoding done, and fcst start')
                encoded_df = set_weight(encoded_df)
                test_lgb, m = one_round_fcst(encoded_df, fcst_date, horizon, features, cate_feats, 
                                             model_params, eval_metric=lgbm_smape) 
                print('%%%%%% fcst done')
                fcst_df = test_lgb.copy()
                fcst_df['index'] = horizon - 1
                res_df_list.append(fcst_df)
                features_list.append(features)
                models.append(m)
        test_lgb = pd.concat(res_df_list).reset_index(drop=True)
        return smape(test_lgb['fcst'], test_lgb['sales_qty'])

if __name__ == "__main__":
    study = optuna.create_study(direction="minimize", pruner = optuna.pruners.MedianPruner(n_warmup_steps = 10))
    study.optimize(objective, n_trials=20)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
# regression
pred_list = []
for class_name in class_list:
    model_params = params[class_name]
    if class_name == 'default':
        feature_class = feature_df
    else:
        feature_class = feature_df[feature_df[model_split_label] == class_name]
    model = LGBModel(objective='regression',
                     feature_name=features,
                     categorical_feature=category_features,
                     feature_df=feature_class,
                     date_col=date_col,
                     target_col=target_col,
                     params=model_params)
    create_models_folder(model_version, class_name)
    model.train_test_split()
    model.train()
    _ = model.predict()
    save_lgb_model(model.model, horizon, model_version, class_name)
    pred_list.append(model.pred_df)
pred_df = pd.concat(pred_list)[output_cols]